In [8]:
import pyproj
import geopandas as gpd
import pandas as pd
import contextily as cx
import matplotlib.pyplot as plt

from segregation.singlegroup import *
from segregation import inference

In [9]:
def seg_profile_mun(shp_path, csv_path, cod_ibge_mun, plot_dinamico = True):
    filepath = shp_path
    gdf = gpd.read_file(filepath)
    gdf_cidade = gdf[gdf['CD_MUN'] == cod_ibge_mun]

    desired_columns = [
        'CD_SETOR', 
        'V01317', 
        'V01318', 
        'V01319', 
        'V01320', 
        'V01321'
    ]
    
    df = pd.read_csv(csv_path, sep = ';', usecols=desired_columns)
    df['CD_SETOR'] = df['CD_SETOR'].astype('str')
    merged_gdf = gdf_cidade.merge(df, on='CD_SETOR', how='left')

    merged_gdf = merged_gdf.to_crs(epsg=3857)

    cols_to_clean = ['V01317', 'V01318', 'V01319', 'V01320', 'V01321']
    merged_gdf[cols_to_clean] = merged_gdf[cols_to_clean].replace('X', 0)
    merged_gdf = merged_gdf.fillna(0)
    merged_gdf[cols_to_clean] = merged_gdf[cols_to_clean].astype('int64')

    merged_gdf['pop_total'] = merged_gdf[cols_to_clean].sum(axis=1)
    
    # Percentual de pretos e pardos
    merged_gdf['pp_total'] = merged_gdf['V01318'] + merged_gdf['V01320']
    merged_gdf['ppp'] = merged_gdf['pp_total'] / merged_gdf['pop_total']

    # Mapa Estático

    # Create the plot and get the axes object
    ax = merged_gdf.to_crs(epsg=3857).plot(
        column='ppp',      # The name of the numeric column to use for coloring
        cmap='YlOrRd',        # The colormap to use (viridis is a good default or YlOrRd)
        legend=True,           # Display the color bar legend
        figsize=(10, 10)        # Optional: control the figure size
    )
    
    # Add the basemap
    cx.add_basemap(ax)
    
    # Optional: Customize or remove the axis for a cleaner map
    # ax.set_axis_off()
    
    plt.show()

    dissim = Dissim(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    s_dissim = SpatialDissim(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    gini = Gini(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    entropy = Entropy(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    isol = Isolation(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    dd_isol = DistanceDecayIsolation(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    conc = RelativeConcentration(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    cent = RelativeCentralization(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')
    clust = RelativeClustering(merged_gdf, group_pop_var='pp_total', total_pop_var='pop_total')

    print(f"Dissimilarity: {dissim.statistic}")
    print(f"Spatial Dissimilarity: {s_dissim.statistic}")
    print(f"Gini: {gini.statistic}")
    print(f"Entropy: {entropy.statistic}")
    print(f"Isolation: {isol.statistic}")
    print(f"Dist. Decay Isolation: {dd_isol.statistic}")
    print(f"Rel. Concentration: {conc.statistic}")
    print(f"Rel. Centralization: {cent.statistic}")
    print(f"Rel. Clustering: {clust.statistic}")

    # Mapa Dinâmico
    # Como são Paulo NÃO TEM oficialmente Bairros mas sim distritos (https://www1.folha.uol.com.br/cotidiano/2026/01/bairros-de-sao-paulo-nao-existem-oficialmente-na-cidade.shtml)
    if plot_dinamico:
        return merged_gdf[['geometry', 'NM_REGIAO', 'NM_DIST', 'NM_SUBDIST', 'NM_BAIRRO', 'ppp']].explore(column="ppp",tooltip=False,popup=True,cmap="YlOrRd",tiles="OpenStreetMap")